In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer ,PreTrainedTokenizerFast
import json
import re
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch

In [3]:
with open("./Similar_data_modify.json","r",encoding="utf-8") as f:
    data = json.load(f)

print(data["data"][0].keys())

dict_keys(['para'])


In [4]:
print(data["data"][0]["para"][0]["label"][0]["token_str"])

산더미


In [5]:
Similar_raw = []

for item in data["data"]:
    for para in item["para"]:
        context = para["context"]
        for labels in para["label"]:
            label = labels["sequence"]
            token_str = labels["token_str"]
            Similar_raw.append((context,label,token_str))

print(len(Similar_raw))

74489


In [6]:
train_data_raw , test_data_raw = train_test_split(Similar_raw,test_size = 0.2)

print(len(train_data_raw),len(test_data_raw))

59591 14898


In [7]:
train_data = pd.DataFrame(train_data_raw)
test_data = pd.DataFrame(test_data_raw)
print(len(train_data))
train_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)
test_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)

train_data.head(5)

59591


,context,label,token
0,그 생각의 그림자와 같이 그의 머리에는 일성이의 모양이 나타났다.,그 생각의 결과 와 같이 그의 머리에는 일성이의 모양이 나타났다.,결과
1,그렇지만 나같이 팔자가 사나워서 이 세상에서도 붙일 곳이 없는 것이 어떻게 극락 왕...,그렇지만 ##그러 같이 팔자가 사그러 워서 이 세상에서도 붙일 곳이 없는 것이 어...,##그러
2,그러나 건실하게 보이는 골격이며 침묵한 태도는 삼십 다 된 사람같이 보이면서도 입 ...,그러나 건실하게 보이는 골격이며 침묵한 태도는 삼십 다 된 나이 같이 보이면서도 입...,나이
3,그리고 부하 장졸 중에서 사람을 골라 군중과 성중을 다스리는 여러 벼슬을 내리고 각...,그리고 부하 장졸 중에서 사람을 골라 군중과 성중을 다스리는 여러 벼슬을 내리고 각...,산더미
4,"떡쇠 모자는 그래도 모서리 빠진 개다리소반이나마 지시로 물 같은 간장 한 종지, 쓰...","떡쇠 모자는 그래도 모서리 빠진 개다리소반이나마 지시로도 같은 간장 한 종지, 쓰디...",##도


In [8]:
test_data.head(5)

,context,label,token
0,순영이는 자기가 한 말같이 전날의 만났던 것을 생각지 않는 것같이 만나는 그 순간순...,순영이는 자기가 한 일과 같이 전날의 만났던 것을 생각지 않는 것같이 만나는 그 순...,일과
1,선종은 소지가 떠난 뒤에는 소허와 같이 말이 적어지고 소허와 같이 순한 사람이 되어...,선종은 소지가 떠난 뒤에는 소 ##소 와 같이 말이 적어지고 소소 와 같이 순한 ...,##소
2,아랫목에서 여러 사람이 크게 웃는 소리가 마치 십 리 밖에서 나는 소리같이 작다랗게...,아랫목에서 여러 사람이 크게 웃는 물소리 가 마치 십 리 밖에서 나는 물소리 같이...,물소리
3,"나는 차디찬 구들에서 스러져 가는 화로를 끼고 아내와 같이 신문을 읽고 있다가, “...","나는 차디찬 구들에서 스러져 가는 화로를 끼고 친구 와 같이 신문을 읽고 있다가,...",친구
4,사실 술처럼 고마운 것은 없었다.,사실 이것 처럼 고마운 것은 없었다.,이것


In [9]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2',)
print(tokenizer.special_tokens_map)

print(tokenizer.tokenize(test_data["context"][0]))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.


{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}
['▁순', '영', '이는', '▁자기가', '▁한', '▁말', '같이', '▁전날', '의', '▁만났', '던', '▁것을', '▁생각', '지', '▁않는', '▁것', '같이', '▁만나는', '▁그', '▁순간', '순', '간을', '▁행', '락', '▁하였', '다.']


In [10]:
BOS = '</s>'#begin of sequence
EOS = '</s>'#end of sequence
MASK = '<mask>'# mask token
SENT = '<unused0>'#문장토큰
PAD = '<pad>'

In [11]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re
#데이터 처리 클래스
class Similar_Dataset(Dataset):
    def __init__(self, chats, max_len=1024):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.bos= BOS
        self.eos = EOS
        self.sent_token = SENT
        self.mask = MASK
        self.pad = PAD
        self.tokenizer = tokenizer
    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한데이터를 차례차례 DataLoader로 넘김
        turn = self._data.iloc[idx]
        l = turn["label"]  # label을 가져온다.
        c = turn["context"]  # 문맥을 가져온다.
        
        c_toked = self.tokenizer.tokenize(self.bos + c + self.eos)
        c_len = len(c_toked)
        
        l_toked = self.tokenizer.tokenize(self.bos + l + self.eos)
        l_len = len(l_toked)

        #최대길이 초과시 처리
        if c_len> self.max_len:
            c_len = self.max_len
            c_toked = c_toked[:c_len]
       
        if l_len> self.max_len:
            l_len = self.max_len
            l_toked = l_toked[:l_len]

        labels_ids = self.tokenizer.convert_tokens_to_ids(l_toked)        
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        decoder_ids = self.tokenizer.convert_tokens_to_ids(l_toked)    
        
        while len(decoder_ids) < self.max_len:
            decoder_ids += [self.tokenizer.pad_token_id]
        
        token_ids = self.tokenizer.convert_tokens_to_ids(c_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, decoder_ids, labels_ids)

In [12]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    decoder_input = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(decoder_input), torch.LongTensor(label)

In [13]:
train_set = Similar_Dataset(train_data, max_len=1024)
loader = DataLoader(train_set, batch_size=4,  num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [14]:
len(loader)

14898

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [16]:
for i, batch in enumerate(loader):
    token_ids, decoder_ids, labels_ids = batch
    print(f"Batch {i+1}")
    print("Token IDs:", token_ids)
    print(len(token_ids[0]))
    print(token_ids.shape)
    print(token_ids[0][7])
    print(tokenizer.decode(token_ids[0][7]))
    print(tokenizer.decode(token_ids[0]))

    print(len(token_ids))
    print("Decoder_ids:", decoder_ids)
    print("Decoder_ids:", (tokenizer.decode(decoder_ids[0])))

    print("Labels IDs:", labels_ids)
    print("Labels IDs:", (tokenizer.decode(labels_ids[0])))
    break

Batch 1
Token IDs: tensor([[    1, 18436, 14040,  ...,     3,     3,     3],
        [    1, 14110, 14027,  ...,     3,     3,     3],
        [    1, 18386, 14075,  ...,     3,     3,     3],
        [    1, 24574, 14426,  ...,     3,     3,     3]])
1024
torch.Size([4, 1024])
tensor(10213)
라
</s> 아무런 기쁜 일도 아무런 쓰라린 일도 다― 통과시키어 전할 수 있는 전신주에 늘어져 있는 전신이야말로 나의 혈관이나 모세관과도 같다고나 할까?」</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [17]:
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
model.to(device)
model.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [18]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

# 모델과 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 스케쥴러 설정
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(loader)*epochs)

# 학습 루프
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(loader, desc="Epoch {:1d}".format(epoch+1), leave=False, disable=False)
    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        input_ids, decoder_input_ids, labels = batch

        # 모델 forward 및 loss 계산
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 백워드 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    print("Epoch {}/{} - Loss: {:.4f}".format(epoch+1, epochs, total_loss / len(loader)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/14898 [00:00<?, ?it/s]

Epoch 1/10 - Loss: 0.0187


Epoch 2/10 - Loss: 0.0061


Epoch 3/10 - Loss: 0.0049


Epoch 4/10 - Loss: 0.0042


...

Epoch 6/10 - Loss: 0.0033


Epoch 7/10 - Loss: 0.0030


Epoch 8/10 - Loss: 0.0028


Epoch 9/10 - Loss: 0.0025


Epoch 10/10 - Loss: 0.0022


In [19]:
model.save_pretrained("./Similar_bart_no_dinput_model1")
tokenizer.save_pretrained("./Similar_bart_no_dinput_model1")

('./Similar_bart_no_dinput_model1/tokenizer_config.json',
 './Similar_bart_no_dinput_model1/special_tokens_map.json',
 './Similar_bart_no_dinput_model1/tokenizer.json')

In [37]:
def generate_qa(context, model, tokenizer, device, num_sequences=2):
    model.eval()  # 모델을 평가 모드로 설정

    # 입력 문맥 토크나이징
    input_ids = tokenizer.encode(BOS + context + EOS, return_tensors="pt").to(device)

    # 모델을 사용하여 출력 생성
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=512,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=2.0,
        do_sample=True,
        num_return_sequences=num_sequences
    )

    generated_sequences = []
    for generated_sequence in output_sequences:
        # 생성된 시퀀스를 디코딩하여 텍스트로 변환
        decoded_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        generated_sequences.append(decoded_text)
    
    return generated_sequences

model_path = "./Similar_bart_no_dinput_model1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def multiple_replace(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

replacements = {SENT: "", BOS: "", EOS: "", PAD: ""}

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = BartForConditionalGeneration.from_pretrained(model_path).to(device)
context = test_data["context"][8]
print("원본:", context)

# context를 input으로 넣으면 context +question +answer 형태로 만듬
generated_sentences = generate_qa(context, model, tokenizer, device, num_sequences=2)

for idx, sentence in enumerate(generated_sentences):
    sentence = multiple_replace(sentence, replacements)
    print(f"생성 {idx+1}:", sentence)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


원본: 이 모양을 보는 명수는, 차마 견디어 볼 수가 없는 듯이, 옆의 사람들을 얼른 돌아보면서, 혼잣말같이 “어떻게 하나…… 암만하여도…… 마지막으로 유언이나 들어보도록 하지!” 의사가 와서, 주사를 한 지 한 오 분쯤 되어서, 희정은 다시 눈을 떴다.
생성 1: 이 모양을 보는 명수는, 차마 견디어 볼 수가 없는 듯이, 옆의 사람들을 얼른 돌아보면서, 한결 같이 “ 어떻게 하나....... 암만하여도...... 마지막으로 유언이나 들어보도록 하지! ” 의사가 와서, 주사를 한 지 한 오 분쯤 되어서, 희정은 다시 눈을 떴다.
생성 2: 이 모양을 보는 명수는, 차마 견디어 볼 수가 없는 듯이, 옆의 사람들을 얼른 돌아보면서, 다 같이 “ 어떻게 하나...... 암만하여도...... 마지막으로 유언이나 들어보도록 하지! ” 의사가 와서, 주사를 한 지 한 오 분쯤 되어서, 희정은 다시 눈을 떴다.
